# Frontend: Gradio UI

1. This frontend provides a simple chat interface.
2. It sends user input to the FastAPI backend and displays the response.

In [1]:
import gradio as gr
import requests
import sys, os 
sys.path.append(os.path.abspath("..")) # add parent folder (week05/frontend) 
from config import BACKEND_URL

# Function to connect Gradio input to backend
def chat_fn(user_input, history):
    try:
        resp = requests.post(BACKEND_URL, json={"message": user_input})
        reply = resp.json().get("response", "[Error: No response]")
    except Exception as e:
        reply = f"[Error: {e}]"
    history.append((user_input, reply))
    return history, ""

# Gradio Chatbot UI
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type your message")
    clear = gr.Button("Clear Chat")

    def user_submit(user_input, history):
        return chat_fn(user_input, history)

    msg.submit(user_submit, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: ([], ""), None, [chatbot, msg])

demo.launch(inline=False)


/opt/anaconda3/envs/cs3249/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/mx/df5x_jdx2g35w5bxy4dmtbqc0000gn/T/ipykernel_73257/3894240138.py:19: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


# Exercise 1: Gradio UI with advanced features

In this exercise, you are given a partially implemented Gradio frontend for the chatbot.
Your task is to extend the UI with advanced features:

### 1.Send parameters to backend

Update chat_fn so that it also sends the temperature and max_tokens values to the backend, not just the message.

Example payload:

{
  "message": "Hello",
  "temperature": 0.7,
  "max_tokens": 200
}


### 2. Save chat history

Complete the save_history function.

It should export the conversation (list of user–bot turns) into a JSON file with a timestamped filename.

### 3. File upload interaction

Complete the ask_about_file function.

When the user uploads a text file, read its contents and send a summary request to the backend (e.g., “Summarize this file: …”).

Append the backend’s reply to the chat history.

In [ ]:
import gradio as gr
import requests
import sys, os 
sys.path.append(os.path.abspath("..")) # add parent folder (week05/frontend) 
from config import BACKEND_URL

# --------------------------
# Chat Function (basic)
# --------------------------
def chat_fn(user_input, history, temperature, max_tokens):
    # Basic request payload
    payload = {
        "message": user_input,
        # TODO: implement parameter 
        # For now we ignore temperature/max_tokens
        # Students will later implement sending these to backend
    }
    try:
        resp = requests.post(BACKEND_URL, json=payload)
        reply = resp.json().get("response", "[Error: No response]")
    except Exception as e:
        reply = f"[Frontend error: {e}]"
    history.append((user_input, reply))
    return history, ""


# --------------------------
# Placeholder Functions (for students to implement)
# --------------------------
def save_history(history):
    # TODO: implement saving history to a file
    return "Saving not implemented yet."

def ask_about_file(file, history):
    # TODO: implement reading file and asking backend a question
    return history


# --------------------------
# UI Layout
# --------------------------
with gr.Blocks() as demo:
    with gr.Row():
        # Left: main chat area
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="CUI Chatbot")
            msg = gr.Textbox(label="Type your message here")
            clear = gr.Button("Clear Chat")

        # Right: sidebar with advanced features
        with gr.Column(scale=1):
            gr.Markdown("## Advanced Options")

            # Controls
            temp = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")
            max_toks = gr.Slider(10, 500, value=200, step=10, label="Max Tokens")

            save_btn = gr.Button("💾 Save History")
            file_upload = gr.File(label="Upload a text file")


    # Chat submit
    msg.submit(chat_fn, [msg, chatbot, temp, max_toks], [chatbot, msg])
    clear.click(lambda: ([], ""), None, [chatbot, msg])

    # Advanced features (currently stubs)
    save_btn.click(save_history, [chatbot], None)
    file_upload.upload(ask_about_file, [file_upload, chatbot], [chatbot])


demo.launch(inline=False)


# Exercise 1 Answer: Gradio UI with advanced features

In [ ]:
import gradio as gr
import requests
from gtts import gTTS
import sys, os 
sys.path.append(os.path.abspath("..")) # add parent folder (week05/frontend) 
from config import BACKEND_URL

# --------------------------
# Chat Function (basic + params)
# --------------------------
def chat_fn(user_input, history, temperature, max_tokens):
    payload = {
        "message": user_input,
        # NOTE: implement parameter 
        "temperature": float(temperature),
        "max_tokens": int(max_tokens)
    }
    try:
        resp = requests.post(BACKEND_URL, json=payload)
        reply = resp.json().get("response", "[Error: No response]")
    except Exception as e:
        reply = f"[Frontend error: {e}]"
    history.append((user_input, reply))
    return history, ""


# --------------------------
# Advanced Features
# --------------------------

def save_history(history):
    """Save chat history to a timestamped JSON file and return UI updates."""
    # NOTE: implement saving history to a file
    import os, json
    from datetime import datetime

    # history is a list of tuples: [(user, bot), ...]
    data = [{"user": u, "bot": b} for (u, b) in history]

    # timestamped filename to avoid overwrite
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"chat_history_{ts}.json"

    with open(fname, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    # return a status message and make the download button point to the new file
    msg = f"✅ Saved to `{fname}` ({len(data)} turns)."
    return msg, gr.update(value=fname, visible=True)



def ask_about_file(file, history):
    """Send uploaded file content to backend with a prompt."""
    # NOTE: implement reading file and asking backend a question
    try:
        text = open(file.name, "r", encoding="utf-8", errors="ignore").read()
        snippet = text[:1000]  # limit size for demo
        question = "Summarize this file:\n" + snippet
        resp = requests.post(BACKEND_URL, json={"message": question})
        reply = resp.json().get("response", "[Error: No response]")
        history.append(("Summarize the uploaded file", reply))
        return history
    except Exception as e:
        history.append(("File upload failed", f"[Error: {e}]"))
        return history


# --------------------------
# UI Layout
# --------------------------
with gr.Blocks() as demo:
    with gr.Row():
        # Left: main chat area
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="CUI Chatbot")
            msg = gr.Textbox(label="Type your message here")
            clear = gr.Button("Clear Chat")

        # Right: sidebar with advanced features
        with gr.Column(scale=1):
            gr.Markdown("## Advanced Options")

            temp = gr.Slider(0, 1, value=0.7, step=0.1, label="Temperature")
            max_toks = gr.Slider(10, 500, value=200, step=10, label="Max Tokens")

            save_btn = gr.Button("💾 Save History")

            # NEW: status text + download button (initially hidden)
            save_status = gr.Markdown("")  
            download_btn = gr.DownloadButton("Download last export", visible=False)

            file_upload = gr.File(label="Upload a text file")


    # Chat submit
    msg.submit(chat_fn, [msg, chatbot, temp, max_toks], [chatbot, msg])
    clear.click(lambda: ([], ""), None, [chatbot, msg])

    # Advanced features
    save_btn.click(save_history, [chatbot], [save_status, download_btn])
    file_upload.upload(ask_about_file, [file_upload, chatbot], [chatbot])

demo.launch(inline=False, inbrowser=True)
